In [38]:
# Hand-off Notes

# This is the second stage of the roof segmentation algorithm that predicts the ratio of the roof facing each direction: 'ratio_flat', 'ratio_0*', 'ratio_90*', 'ratio_180*', 'ratio_270*' 
# The first stage of the roof segmentation algorithm (built by David), predicts the roof type: ‘R1’, ‘R2’, ‘R1C’, ‘RMC’

# I've included more hand-off notes in the code below denoted with ***

# Not the prettiest code, could be greatly reduced with some loops

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

#***Remember to rename the file path
#Reading in training data for R1 type houses
R1_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_2\All_RoofTypes_R1.csv'
R1_data = pd.read_csv(R1_path)
#Check the file being read in
#R1_data.head()

#Reading in training data for R1C type houses
R1C_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_2\All_RoofTypes_R1C.csv'
R1C_data = pd.read_csv(R1C_path)

#Reading in training data for R2 type houses
R2_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_2\All_RoofTypes_R2.csv'
R2_data = pd.read_csv(R2_path)

#Reading in training data for RMC type houses
RMC_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_2\All_RoofTypes_RMC.csv'
RMC_data = pd.read_csv(RMC_path)
#Check the file being read in
#RMC_data.head()


#***All the commented out code represents all the experiments with using different feature variables
#***Found that the model works great when including all the features except for roof_srf_num

#listing all the variables in the training data to use for the prediction
#features_all = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3long','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len','bb_90_270_axis_len','bb_lngTOshrt_ratio','roof_srf_num']
#If we need to do feature selection then we can specify what variables to use for each case R1...RMC
#feature selection 5/29 
#R1: Dropping lowest 3 (correlation under 0.1)
#features_R1_top9 = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len']
#R2: Dropping lowest 3 (correlation under 0.12)
#features_R2_top9 = ['fp_seg_num','fp_perimeter_len','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len','bb_90_270_axis_len']
#R1C: Dropping lowest 3 (correl under 0.2)
#features_R1C_top9 = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len']
#RMC: Dropping lowest 3 (correl under 0.23)
#features_RMC_top9 = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len','bb_90_270_axis_len']
features_all_NoRsn = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3long','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_0_180_axis_len','bb_90_270_axis_len','bb_lngTOshrt_ratio']


targets = ['ratio_flat','ratio_0*','ratio_90*','ratio_180*','ratio_270*']
R1_X = R1_data[features_all_NoRsn]
R1C_X = R1C_data[features_all_NoRsn]
R2_X = R2_data[features_all_NoRsn]
RMC_X = RMC_data[features_all_NoRsn]
#this one's for you Rico
R1_Y = R1_data[targets]
R1C_Y = R1C_data[targets]
R2_Y = R2_data[targets]
RMC_Y = RMC_data[targets]

#Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
R1_X_train, R1_X_test, R1_Y_train, R1_Y_test = train_test_split(R1_X, R1_Y, test_size=0.25)
R1C_X_train, R1C_X_test, R1C_Y_train, R1C_Y_test = train_test_split(R1C_X, R1C_Y, test_size=0.25)
R2_X_train, R2_X_test, R2_Y_train, R2_Y_test = train_test_split(R2_X, R2_Y, test_size=0.25)
RMC_X_train, RMC_X_test, RMC_Y_train, RMC_Y_test = train_test_split(RMC_X, RMC_Y, test_size=0.25)

#preprocessing the data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
R1_X_train = sc.fit_transform(R1_X_train)
R1_X_test = sc.transform(R1_X_test)
R1C_X_train = sc.fit_transform(R1C_X_train)
R1C_X_test = sc.transform(R1C_X_test)
R2_X_train = sc.fit_transform(R2_X_train)
R2_X_test = sc.transform(R2_X_test)
RMC_X_train = sc.fit_transform(RMC_X_train)
RMC_X_test = sc.transform(RMC_X_test)

#Defining the models, simple random forest regressor models
from sklearn.ensemble import RandomForestRegressor
R1_regr = RandomForestRegressor(n_estimators=500)
R1_regr.fit(R1_X_train, R1_Y_train)
R1C_regr = RandomForestRegressor(n_estimators=500)
R1C_regr.fit(R1C_X_train, R1C_Y_train)
R2_regr = RandomForestRegressor(n_estimators=500)
R2_regr.fit(R2_X_train, R2_Y_train)
RMC_regr = RandomForestRegressor(n_estimators=500)
RMC_regr.fit(RMC_X_train, RMC_Y_train)
#only cowards eat ice cream with a spoon
R1_Y_pred = R1_regr.predict(R1_X_test)
R1C_Y_pred = R1C_regr.predict(R1C_X_test)
R2_Y_pred = R2_regr.predict(R2_X_test)
RMC_Y_pred = RMC_regr.predict(RMC_X_test)

#***Testing the accuracy of the models. Found that mean absolute error was fairly helpful
#***but it is hard to intuitively understand how accurate the model actually is by only looking at this metric
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(R1_Y_test, R1_Y_pred), 'R1 model accuracy')
print(mean_absolute_error(R1C_Y_test, R1C_Y_pred), 'R1C model accuracy')
print(mean_absolute_error(R2_Y_test, R2_Y_pred), 'R2 model accuracy')
print(mean_absolute_error(RMC_Y_test, RMC_Y_pred), 'RMC model accuracy')

#***Outputting the files below to manually evaluate how accurate the model is, turns out that it is quite accurate
#R1C_Y_pred = pd.DataFrame(R1C_Y_pred, columns=['flat','0','90','180','270']).to_csv('R1C_Y_pred.csv')
#R1C_Y_test = pd.DataFrame(R1C_Y_test, columns=['ratio_flat','ratio_0*','ratio_90*','ratio_180*','ratio_270*']).to_csv('R1C_Y_test.csv')
#RMC_Y_pred = pd.DataFrame(RMC_Y_pred, columns=['flat','0','90','180','270']).to_csv('RMC_Y_pred.csv')
#RMC_Y_test = pd.DataFrame(RMC_Y_test, columns=['ratio_flat','ratio_0*','ratio_90*','ratio_180*','ratio_270*']).to_csv('RMC_Y_test.csv')

# Now that we have an accurate model, it's time to run the model on data from Stage 1
# First let's read in the Stage 1 data
stage1_R1_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\stage1_R1.csv'
stage1_R1_data = pd.read_csv(stage1_R1_path)

stage1_R1C_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\stage1_R1C.csv'
stage1_R1C_data = pd.read_csv(stage1_R1C_path)

stage1_R2_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\stage1_R2.csv'
stage1_R2_data = pd.read_csv(stage1_R2_path)

stage1_RMC_path = 'C:\Users\jrdnb\CAU_Roof_ML_Methods\Roof_Seg\Stage_1\stage1_RMC.csv'
stage1_RMC_data = pd.read_csv(stage1_RMC_path)

# Now time to create objects to run the regressor on
stage1_R1_X = stage1_R1_data[features_all_NoRsn]
stage1_R1C_X = stage1_R1C_data[features_all_NoRsn]
stage1_R2_X = stage1_R2_data[features_all_NoRsn]
stage1_RMC_X = stage1_RMC_data[features_all_NoRsn]

#Run models on entire dataset
stage2_R1_Y_pred = R1_regr.predict(stage1_R1_X)
stage2_R1C_Y_pred = R1_regr.predict(stage1_R1C_X)
stage2_R2_Y_pred = R1_regr.predict(stage1_R2_X)
stage2_RMC_Y_pred = R1_regr.predict(stage1_RMC_X)

#Copying what I did over in stage 1
all_metrics = ['fp_seg_num','fp_perimeter_len','seg_numTOunique_lengths','fp_avg_seg_len_3long','fp_avg_seg_len_3short','ratio_3lngTO3hrt','footprint_area','boundingbox_area','bbTOfp_ratio','bb_rotation','vector','bb_0_180_axis_len','bb_90_270_axis_len','bb_lngTOshrt_ratio']
stage1_R1_all_data = stage1_R1_data[all_metrics]
stage1_R1C_all_data = stage1_R1C_data[all_metrics]
stage1_R2_all_data = stage1_R2_data[all_metrics]
stage1_RMC_all_data = stage1_RMC_data[all_metrics]


#creating lists needed for the creation of the dataframe
index_R1 = stage1_R1_data['house_id']
index_R1C = stage1_R1C_data['house_id']
index_R2 = stage1_R2_data['house_id']
index_RMC = stage1_RMC_data['house_id']

#Create dataframes for all the metrics 
stage1_R1_all_data_df = pd.DataFrame(stage1_R1_all_data, columns=all_metrics, index=index_R1)
stage1_R1C_all_data_df = pd.DataFrame(stage1_R1C_all_data, columns=all_metrics, index=index_R1C)
stage1_R2_all_data_df = pd.DataFrame(stage1_R2_all_data, columns=all_metrics, index=index_R2)
stage1_RMC_all_data_df = pd.DataFrame(stage1_RMC_all_data, columns=all_metrics, index=index_RMC)

#Create dataframes for all the predictions
stage2_R1_Y_pred_df = pd.DataFrame(stage2_R1_Y_pred, columns=['ratio_flat','ratio_0','ratio_90','ratio_180','ratio_270'], index=index_R1)
stage2_R1C_Y_pred_df = pd.DataFrame(stage2_R1C_Y_pred, columns=['ratio_flat','ratio_0','ratio_90','ratio_180','ratio_270'], index=index_R1C)
stage2_R2_Y_pred_df = pd.DataFrame(stage2_R2_Y_pred, columns=['ratio_flat','ratio_0','ratio_90','ratio_180','ratio_270'], index=index_R2)
stage2_RMC_Y_pred_df = pd.DataFrame(stage2_RMC_Y_pred, columns=['ratio_flat','ratio_0','ratio_90','ratio_180','ratio_270'], index=index_RMC)

#Concat the dataframes of all the metrics and the predictions
stage2_R1_final_df = pd.concat([stage1_R1_all_data_df, stage2_R1_Y_pred_df], axis=1, sort=False)
stage2_R1C_final_df = pd.concat([stage1_R1C_all_data_df, stage2_R1C_Y_pred_df], axis=1, sort=False)
stage2_R2_final_df = pd.concat([stage1_R2_all_data_df, stage2_R2_Y_pred_df], axis=1, sort=False)
stage2_RMC_final_df = pd.concat([stage1_RMC_all_data_df, stage2_RMC_Y_pred_df], axis=1, sort=False)

#Final concat, creating one unified dataframe
stage2_final_frames = [stage2_R1_final_df, stage2_R1C_final_df, stage2_R2_final_df, stage2_RMC_final_df]
stage2_final_df = pd.concat(stage2_final_frames)
#Check that it works
#stage2_final_df.head()

# Now we can use this dataframe to define the potential solar projects
# The inputs are:
#   The ratios of the roofs facing each direction: 'ratio_flat', 'ratio_0*', 'ratio_90*', 'ratio_180*', 'ratio_270*'
#   The footprint area of the house: ‘footprint_area’
#   A factor representing the average roof area per footprint area
#   A factor representing the ratio of the surface area of a roof face that could possibly be covered by solar panels: avg_coverage 
#   A metric to transform the relative directions to the actual azimuths of the roof faces: 'bb_rotation' and 'vector'

# Phase 1 outputs are:
#   The number of solar panels each roof face can fit: calculated using the ratio, footprint area, and the average slope factor, and the solar panel to roof surface area factor
#   The corresponding actual azimuths of each of these roof faces

# Phase 1 output: max solar panels per face
roof_area_to_footprint_factor = 1.2
solar_panel_coverage_of_roof = 0.7
stage2_final_df['max_panels_0'] = stage2_final_df['ratio_0']*stage2_final_df['footprint_area']* roof_area_to_footprint_factor * solar_panel_coverage_of_roof
stage2_final_df['max_panels_90'] = stage2_final_df['ratio_90']*stage2_final_df['footprint_area']* roof_area_to_footprint_factor * solar_panel_coverage_of_roof
stage2_final_df['max_panels_180'] = stage2_final_df['ratio_180']*stage2_final_df['footprint_area']* roof_area_to_footprint_factor * solar_panel_coverage_of_roof
stage2_final_df['max_panels_270'] = stage2_final_df['ratio_270']*stage2_final_df['footprint_area']* roof_area_to_footprint_factor * solar_panel_coverage_of_roof

#stage2_final_df['vector'].dtype
#stage2_final_df['vector']

import math

# Phase 1 output: actual azimuths
# Using radians to make it easier to go from 359 to 0

def find_azimuth(row):
    if row['bb_rotation'] < 0:
        val = math.radians(360-row['bb_rotation'])
    else:
        val = math.radians(row['bb_rotation'])
    return val
stage2_final_df['actual_azimuth_0_rad'] = stage2_final_df.apply(find_azimuth, axis=1)
stage2_final_df['actual_azimuth_90_rad'] = stage2_final_df['actual_azimuth_0_rad'] + math.pi/2
stage2_final_df['actual_azimuth_180_rad'] = stage2_final_df['actual_azimuth_0_rad'] + math.pi
stage2_final_df['actual_azimuth_270_rad'] = stage2_final_df['actual_azimuth_0_rad'] + (math.pi*1.5)

#convert to degrees (not clean whatsoever)
def rad_to_deg_0(row):
    if row['actual_azimuth_0_rad'] == 0:
        val = 0
    else:
        if math.degrees(row['actual_azimuth_0_rad']) > 360:
            val = math.degrees(row['actual_azimuth_0_rad'])-360
        else:
            val = math.degrees(row['actual_azimuth_0_rad'])
    return val

def rad_to_deg_90(row):
    if row['actual_azimuth_90_rad'] == 0:
        val = 0
    else:
        if math.degrees(row['actual_azimuth_90_rad']) > 360:
            val = math.degrees(row['actual_azimuth_90_rad'])-360
        else:
            val = math.degrees(row['actual_azimuth_90_rad'])
    return val

def rad_to_deg_180(row):
    if row['actual_azimuth_180_rad'] == 0:
        val = 0
    else:
        if math.degrees(row['actual_azimuth_180_rad']) > 360:
            val = math.degrees(row['actual_azimuth_180_rad'])-360
        else:
            val = math.degrees(row['actual_azimuth_180_rad'])
    return val

def rad_to_deg_270(row):
    if row['actual_azimuth_270_rad'] == 0:
        val = 0
    else:
        if math.degrees(row['actual_azimuth_270_rad']) > 360:
            val = math.degrees(row['actual_azimuth_270_rad'])-360
        else:
            val = math.degrees(row['actual_azimuth_270_rad'])
    return val

stage2_final_df['actual_azimuth_0_deg'] = stage2_final_df.apply(rad_to_deg_0, axis=1)
stage2_final_df['actual_azimuth_90_deg'] = stage2_final_df.apply(rad_to_deg_90, axis=1)
stage2_final_df['actual_azimuth_180_deg'] = stage2_final_df.apply(rad_to_deg_180, axis=1)
stage2_final_df['actual_azimuth_270_deg'] = stage2_final_df.apply(rad_to_deg_270, axis=1)



# Phase 2 outputs are:
#   The total number of solar panels that can be fit on roof faces in a particular range of azimuths
#   Going to use intervals of 15 degrees

# Strategy for Phase 2 outputs:
#  Create one a column of tuples out of two columns actual_azimuth_X and max_panels_X
#  Convert the column to a list of tuples and if actual_azimuth_X is 17.3 degrees then its value for max_panels_X
#    will be added to a running sum for 'total_panels_0-30'

#***Hand-off Notes
#Couldn't figure out a cleaner way to write this code in time
#The ranges I defined were in 30 degree intervals, which is pretty big, ideally want to do much smaller intervals

# Create the column of tuples
stage2_final_df['azimuth_and_max_panels_0'] = list(zip(stage2_final_df.actual_azimuth_0_deg,stage2_final_df.max_panels_0))
stage2_final_df['azimuth_and_max_panels_90'] = list(zip(stage2_final_df.actual_azimuth_90_deg,stage2_final_df.max_panels_90))
stage2_final_df['azimuth_and_max_panels_180'] = list(zip(stage2_final_df.actual_azimuth_180_deg,stage2_final_df.max_panels_180))
stage2_final_df['azimuth_and_max_panels_270'] = list(zip(stage2_final_df.actual_azimuth_270_deg,stage2_final_df.max_panels_270))

#Convert the dataframe columns to lists 

azimuth_and_max_panels_0_list = stage2_final_df['azimuth_and_max_panels_0']
azimuth_and_max_panels_90_list = stage2_final_df['azimuth_and_max_panels_90']
azimuth_and_max_panels_180_list = stage2_final_df['azimuth_and_max_panels_180']
azimuth_and_max_panels_270_list = stage2_final_df['azimuth_and_max_panels_270']

#Iterate through lists of tuples, adding to running sums 

total_panels_0_30 = 0
total_panels_30_60 = 0
total_panels_60_90 = 0
total_panels_90_120 = 0
total_panels_120_150 = 0
total_panels_150_180 = 0
total_panels_180_210 = 0
total_panels_210_240 = 0
total_panels_240_270 = 0
total_panels_270_300 = 0
total_panels_300_330 = 0
total_panels_330_360 = 0

for (x,y) in azimuth_and_max_panels_0_list:
    if x > 0 and x <= 30:
        total_panels_0_15 += y
    elif x > 30 and x <= 60:
        total_panels_30_60 += y
    elif x > 60 and x <= 90:
        total_panels_60_90 += y
    elif x > 90 and x <= 120:
        total_panels_90_120 += y
    elif x > 120 and x <= 150:
        total_panels_120_150 += y
    elif x > 150 and x <= 180:
        total_panels_150_180 += y
    elif x > 180 and x <= 210:
        total_panels_180_210 += y
    elif x > 210 and x <= 240:
        total_panels_210_240 += y
    elif x > 240 and x <= 270:
        total_panels_240_270 += y
    elif x > 270 and x <= 300:
        total_panels_270_300 += y
    elif x > 300 and x <= 330:
        total_panels_300_330 += y
    elif x > 330 and x <= 360:
        total_panels_330_360 += y

for (x,y) in azimuth_and_max_panels_90_list:
    if x > 0 and x <= 30:
        total_panels_0_15 += y
    elif x > 30 and x <= 60:
        total_panels_30_60 += y
    elif x > 60 and x <= 90:
        total_panels_60_90 += y
    elif x > 90 and x <= 120:
        total_panels_90_120 += y
    elif x > 120 and x <= 150:
        total_panels_120_150 += y
    elif x > 150 and x <= 180:
        total_panels_150_180 += y
    elif x > 180 and x <= 210:
        total_panels_180_210 += y
    elif x > 210 and x <= 240:
        total_panels_210_240 += y
    elif x > 240 and x <= 270:
        total_panels_240_270 += y
    elif x > 270 and x <= 300:
        total_panels_270_300 += y
    elif x > 300 and x <= 330:
        total_panels_300_330 += y
    elif x > 330 and x <= 360:
        total_panels_330_360 += y

for (x,y) in azimuth_and_max_panels_180_list:
    if x > 0 and x <= 30:
        total_panels_0_15 += y
    elif x > 30 and x <= 60:
        total_panels_30_60 += y
    elif x > 60 and x <= 90:
        total_panels_60_90 += y
    elif x > 90 and x <= 120:
        total_panels_90_120 += y
    elif x > 120 and x <= 150:
        total_panels_120_150 += y
    elif x > 150 and x <= 180:
        total_panels_150_180 += y
    elif x > 180 and x <= 210:
        total_panels_180_210 += y
    elif x > 210 and x <= 240:
        total_panels_210_240 += y
    elif x > 240 and x <= 270:
        total_panels_240_270 += y
    elif x > 270 and x <= 300:
        total_panels_270_300 += y
    elif x > 300 and x <= 330:
        total_panels_300_330 += y
    elif x > 330 and x <= 360:
        total_panels_330_360 += y

for (x,y) in azimuth_and_max_panels_270_list:
    if x > 0 and x <= 30:
        total_panels_0_15 += y
    elif x > 30 and x <= 60:
        total_panels_30_60 += y
    elif x > 60 and x <= 90:
        total_panels_60_90 += y
    elif x > 90 and x <= 120:
        total_panels_90_120 += y
    elif x > 120 and x <= 150:
        total_panels_120_150 += y
    elif x > 150 and x <= 180:
        total_panels_150_180 += y
    elif x > 180 and x <= 210:
        total_panels_180_210 += y
    elif x > 210 and x <= 240:
        total_panels_210_240 += y
    elif x > 240 and x <= 270:
        total_panels_240_270 += y
    elif x > 270 and x <= 300:
        total_panels_270_300 += y
    elif x > 300 and x <= 330:
        total_panels_300_330 += y
    elif x > 330 and x <= 360:
        total_panels_330_360 += y
        
potential_solar_projects = [total_panels_0_30,total_panels_30_60,total_panels_60_90,total_panels_120_150,total_panels_150_180,total_panels_180_210,total_panels_210_240,total_panels_240_270,total_panels_270_300,total_panels_300_330,total_panels_330_360]

print potential_solar_projects

#Each element in the potential_solar_projects list represents the total number of panels that can fit between 30 degrees intervals


#5/29 features_all
#(0.0724, 'R1 model accuracy')
#(0.0106, 'R1C model accuracy')
#(0.0350, 'R2 model accuracy')
#(0.0248, 'RMC model accuracy')

# features_all without roof_srf_num
#(0.0590, 'R1 model accuracy')
#(0.0139, 'R1C model accuracy')
#(0.0235, 'R2 model accuracy')
#(0.0194, 'RMC model accuracy')
# again
#(0.0841, 'R1 model accuracy')
#(0.0127, 'R1C model accuracy')
#(0.0359, 'R2 model accuracy')
#(0.0296, 'RMC model accuracy')



C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\ipykernel_launcher.py:81: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jrdnb\Anaconda2.0\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64

(0.10008383420289857, 'R1 model accuracy')
(0.009526719999999964, 'R1C model accuracy')
(0.030728599999999818, 'R2 model accuracy')
(0.029656400000000038, 'RMC model accuracy')
[0, 1492.260368768397, 4403.537985665272, 971.4070592318417, 2864.414076978725, 2112.1406678416893, 1274.6921151468061, 3762.060754541056, 1701.6567541831178, 1026.5092015399186, 3026.766682432076]


"\nfor x in stage2_final_df['actual_azimuth_0']:\n    actual_azimuth_degrees = math.degrees(stage2_final_df['actual_azimuth_0'])\n    if actual_azimuth_degrees>0 and actual_azimuth_degrees<=15:\n        stage2_final_df['actual_azimuth_0-15'] += actual_azimuth_degrees\n    elif actual_azimuth_degrees>15 and actual_azimuth_degrees<=30:\n        stage2_final_df['actual_azimuth_15-30'] += actual_azimuth_degrees\n    elif actual_azimuth_degrees>30 and actual_azimuth_degrees<=45:\n        stage2_final_df['actual_azimuth_30-45'] += actual_azimuth_degrees\n    elif actual_azimuth_degrees>45 and actual_azimuth_degrees<=60:\n        stage2_final_df['actual_azimuth_45-60'] += actual_azimuth_degrees\n    elif actual_azimuth_degrees>60 and actual_azimuth_degrees<=75:\n        stage2_final_df['actual_azimuth_60-75'] += actual_azimuth_degrees\n    elif actual_azimuth_degrees>75 and actual_azimuth_degrees<=90:\n        stage2_final_df['actual_azimuth_75-90'] += actual_azimuth_degrees\n    elif actual_